In [ ]:
"""
count seems to decrease score
full interactions seems too much

features, shape, iter, cv, public

"""

In [11]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
import gini
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

selected = np.random.rand(train.shape[0]) < 0.8

## --------------------- Feature Start -----------------------

In [7]:
# raw features
ID = test.Id.values

categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numerical = set(train.columns).difference(categorical + ['Hazard', 'Id'])
numerical = list(numerical)

train_c = train.copy()
test_c = test.copy()

In [8]:
%%time
# normalize features
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer

# normalize numerical features
SS = StandardScaler()
train_c[numerical] = SS.fit_transform(train_c[numerical])
test_c[numerical] = SS.transform(test_c[numerical])

# digitize categorical features
DV = DictVectorizer(sparse=False)
train_c = DV.fit_transform(train_c[numerical+categorical].T.to_dict().values())
test_c = DV.transform(test_c[numerical+categorical].T.to_dict().values())

print(train_c.shape)

(50999, 111)
CPU times: user 7.83 s, sys: 60 ms, total: 7.89 s
Wall time: 7.89 s


/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [6]:
%%time
from sklearn.preprocessing import PolynomialFeatures

PF = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
train_c = PF.fit_transform(train_c)
print(train_c.shape)

(50999, 6216)
CPU times: user 6.03 s, sys: 2.56 s, total: 8.59 s
Wall time: 8.59 s


## --------------------- Feature End -----------------------

In [83]:
%%time
from sklearn.linear_model import SGDRegressor
from sklearn.cross_validation import cross_val_score
import gini
from sklearn.metrics import mean_squared_error

gini.normalized_gini??

regressor = SGDRegressor(loss='squared_loss', penalty='l1', alpha=1E-7, n_iter=10)
scores = cross_val_score(regressor, train_c, train.Hazard.values, cv=20, scoring=gini.normalized_gini_score)
print(scores.mean())

-0.00621682827474


In [15]:
%%time
# 0.36597
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

RFR = RandomForestRegressor(n_estimators=10000, max_features='sqrt', min_samples_leaf=20, oob_score=True, n_jobs=-1)
scores = cross_val_score(RFR, train_c, train.Hazard.values, cv=20, scoring=gini.normalized_gini_score, n_jobs=-1)
print(scores.mean())

KeyboardInterrupt: 

In [77]:
model = 'linear1'
yhat = np.loadtxt('../tmp/{}/out'.format(model))
pd.DataFrame({'Id': ID, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/lin01.csv', index=0)

## --------------------- Factory -----------------------

In [75]:
%%time
# kmeans features

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

kmeans = KMeans(80, random_state=12345, n_init=1)
feat_kmeans_train = kmeans.fit_transform(SS.fit_transform(train[numerical]))
feat_kmeans_test = kmeans.transform(SS.transform(test[numerical]))

# print('variance lost:')
# print(kmeans.inertia_ / KMeans(1, random_state=12345, n_init=1).fit(train[numerical]).inertia_)

feat_kmeans_train = pd.DataFrame(feat_kmeans_train).add_prefix('kmeans_')
feat_kmeans_test = pd.DataFrame(feat_kmeans_test).add_prefix('kmeans_')

numerical += feat_kmeans_train.columns.values.tolist()
train_c = pd.concat([train_c, feat_kmeans_train], axis=1)
test_c = pd.concat([test_c, feat_kmeans_test], axis=1)

CPU times: user 14.5 s, sys: 4 ms, total: 14.5 s
Wall time: 14.5 s


/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [49]:
# count features

feat_count_train = train[categorical].copy()
feat_count_test = test[categorical].copy()

for cat in categorical:
    numerical.append('cnt_'+cat)
    hotDeck = train[cat].value_counts().reset_index().rename(columns={'index': cat, 0: 'cnt_'+cat})
    feat_count_train = pd.merge(feat_count_train, hotDeck, on=cat)
    del feat_count_train[cat]
    
    feat_count_test = pd.merge(feat_count_test, hotDeck, on=cat)
    del feat_count_test[cat]
    
train_c = pd.concat([train_c, feat_count_train], axis=1)
test_c = pd.concat([test_c, feat_count_test], axis=1)

In [53]:
# categorical encoding
from sklearn.preprocessing import StandardScaler

feat_encode_train = train[categorical].copy()
feat_encode_test = test[categorical].copy()

for cat in categorical:
    hot_deck = train.groupby(cat).Hazard.mean().reset_index().rename(columns={'Hazard': 'encode_'+cat})
    feat_encode_train = pd.merge(feat_encode_train, hot_deck, on=cat)
    del feat_encode_train[cat]
    
    feat_encode_test = pd.merge(feat_encode_test, hot_deck, on=cat)
    del feat_encode_test[cat]
    
names = feat_encode_train.columns
ss = StandardScaler()
feat_encode_train[names] = ss.fit_transform(feat_encode_train.values)
feat_encode_test[names] = ss.transform(feat_encode_test.values)

encode_T1_V4     1.011131
encode_T1_V5     1.003089
encode_T1_V6     0.999460
encode_T1_V7     1.025445
encode_T1_V8     0.985815
encode_T1_V9     1.000703
encode_T1_V11    1.001235
encode_T1_V12    0.987670
encode_T1_V15    1.007230
encode_T1_V16    1.005126
encode_T1_V17    1.003132
encode_T2_V3     0.998128
encode_T2_V5     1.001219
encode_T2_V11    1.002282
encode_T2_V12    1.004119
encode_T2_V13    0.994556
dtype: float64

In [81]:
%%time
# reduce
from xgboost import XGBRegressor
import xgboost as xgb

feat_reduce_train = pd.DataFrame()
feat_reduce_test = pd.DataFrame()

for num in numerical:
    dTmp = xgb.DMatrix(train[[num]], label=train.Hazard)
    reg = xgb.train({'max_depth':8, 'min_child_weight':30, 'objective':'reg:linear'}, dTmp, 1)
    
    feat_reduce_train['discrete_'+num] = reg.predict(dTmp, pred_leaf=True)
    feat_reduce_train['discrete_'+num] = feat_reduce_train['discrete_'+num].map(str)
    
    feat_reduce_test['discrete_'+num] = reg.predict(xgb.DMatrix(test[[num]]), pred_leaf=True)
    feat_reduce_test['discrete_'+num] = feat_reduce_test['discrete_'+num].map(str)

Wall time: 6.86 s


In [73]:
# discrete

feat_discrete_train = train[numerical].applymap(str)
feat_discrete_test = test[numerical].applymap(str)

In [88]:
# combine all features
cTrain = train[numerical+categorical]
cTest = test[numerical+categorical]

# cTrain = pd.concat([train[numerical+categorical], feat_reduce_train], axis=1)
# cTest = pd.concat([test[numerical+categorical], feat_reduce_test], axis=1)